In [1]:
using Pkg

In [12]:
ENV["CPLEX_STUDIO_BINARIES"] = "C:\\Program Files\\IBM\\ILOG\\CPLEX_Studio_Community129\\cplex\\bin\\x64_win64"

"C:\\Program Files\\IBM\\ILOG\\CPLEX_Studio_Community129\\cplex\\bin\\x64_win64"

In [13]:
Pkg.build("CPLEX")

   Building MbedTLS → `C:\Users\Geoffroy Leconte\.julia\packages\MbedTLS\a1JFn\deps\build.log`
   Building CPLEX ──→ `C:\Users\Geoffroy Leconte\.julia\packages\CPLEX\Hxpuk\deps\build.log`


In [1]:
using CPLEX
using QPSReader
using QuadraticModels
using NLPModels
using SolverTools
using SolverBenchmark
using DataFrames

┌ Info: Precompiling CPLEX [a076750e-1247-5638-91d2-ce28b192dca0]
└ @ Base loading.jl:1260


In [26]:
function createQuadraticModel(qpdata)
    # probleme du point initial
    #x0 = init_x0(qpdata.lvar, qpdata.uvar)
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0)

end

function display_results_CPlex(xpress_model)
    # show results
    sol = CPLEX.get_solution(model_cplex)
    println("soln = $(sol)")

    objv = CPLEX.get_objval(model_cplex)
    println("objv = $(objv)")
end
    

function OptimizeCPlex(qpdata)
    QM = createQuadraticModel(qpdata)
    SM = SlackModel(QM)
    
    env = CPLEX.Env()
    
    model = CPLEX.cplex_model(env; f = grad(SM, zeros(length(SM.meta.x0))),
                Aeq = jac(SM, SM.meta.x0), beq = SM.meta.lcon, 
                lb = SM.meta.lvar, ub = SM.meta.uvar)
    # H = ... if quadratic problem
     # run optimization
    CPLEX.optimize!(model)
    
    if model.terminator == [0]
        ExStats_status = :acceptable
    end
    
    GenExStats = GenericExecutionStats(ExStats_status, SM, solution = CPLEX.get_solution(model),
                               objective = CPLEX.get_objval(model), iter = Int64(CPLEX.c_api_getitcnt(model)),
                                multipliers = CPLEX.get_constr_duals(model))
    println(SM.meta.uvar)
    println(GenExStats)
    
    return model
end

OptimizeCPlex (generic function with 1 method)

In [27]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib"
afiro = string(path_pb, "\\AFIRO.SIF")
qpdata2 = readqps(afiro);

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [28]:
model_cplex = OptimizeCPlex(qpdata2)
display_results_CPlex(model_cplex)

[Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, 80.0, 0.0, 80.0, 0.0, 0.0, 0.0, 500.0, 0.0, 500.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 310.0, 300.0]
Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: -464.7531428571428
  primal feasibility: Inf
  dual feasibility: Inf
  solution: [80.0  25.5  54.5  84.80000000000001 ⋯ 19.30714285714286]
  multipliers: [-0.6285714285714286  0.0  -0.34477142857142856  -0.2285714285714286 ⋯ -0.0]
  iterations: 7
  elapsed time: Inf
soln = [80.0, 25.5, 54.5, 84.80000000000001, 18.214285714285715, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 18.214285714285715, 0.0, 19.30714285714286, 500.0, 475.92, 24.08, 0.0, 215.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 339.9428571428572, 383.9428571428572, 0.0, 0.0, 80.0, 0.0, 18.214285714285715, 0.0, -0.0, -0.0, 500.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, -378.

In [14]:
SM1 = SlackModel(createQuadraticModel(qpdata2))
GenExStats = GenericExecutionStats(:acceptable, SM1, solution = CPLEX.get_solution(model_cplex),
                               objective = CPLEX.get_objval(model_cplex),
                                iter = Int64(CPLEX.c_api_getitcnt(model_cplex)))

"Execution stats: solved to within acceptable tolerances"

In [19]:
CPLEX.get_varLB(model_cplex)

51-element Array{Float64,1}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
 -1.0e20
 -1.0e20
 -1.0e20
 -1.0e20
 -1.0e20
 -1.0e20
 -1.0e20
 -1.0e20
 -1.0e20
 -1.0e20
 -1.0e20
 -1.0e20

In [58]:
model_cplex.terminator == [0]

true

In [30]:
T = typeof(model_cplex)
for (name, typ) in zip(fieldnames(T), T.types)
    println("type of the fieldname $name is $typ")
end

type of the fieldname env is CPLEX.Env
type of the fieldname lp is Ptr{Nothing}
type of the fieldname has_int is Bool
type of the fieldname has_qc is Bool
type of the fieldname has_sos is Bool
type of the fieldname callback is Any
type of the fieldname terminator is Array{Int32,1}


In [27]:
CPLEX.get_status(model_cplex)

:CPX_STAT_OPTIMAL

In [42]:
CPLEX.get_constr_duals(model_cplex)

27-element Array{Float64,1}:
 -0.6285714285714286
  0.0
 -0.34477142857142856
 -0.2285714285714286
 -0.0
  0.0
 -0.0
 -2.249657142857143
 -0.0
 -0.0
 -0.9428571428571428
  0.0
 -0.8743428571428571
  ⋮
  0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.9428571428571428
 -0.6285714285714286
 -0.0
 -0.9428571428571428
 -0.0
 -0.0
 -0.0

In [31]:
T

CPLEX.Model